### 이미지 처리
1. TextGrid를 연다 
2. interval을 가져온다 
3. xmin/xmax를 가져온다 
4. frame_rate에 따라 start_frame, end_frame을 구한다. 

### face: mouth_embedding 
### VAfeature_256
### emotion

In [1]:
import os
import torch
import random
import json
import tgt
import numpy as np
import pyworld as pw
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

from scipy.io.wavfile import read
import librosa
from librosa.filters import mel as librosa_mel_fn
import torch
import torchaudio
import glob
# 환경 변수 설정
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
print("CUDA_VISIBLE_DEVICES:", os.environ['CUDA_VISIBLE_DEVICES'])

# CUDA 사용 가능 여부와 디바이스 수 확인
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Device Count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Current CUDA Device: {torch.cuda.current_device()}")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print(torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(0))

MAX_WAV_VALUE = 32768.0

print(torch.__version__)
print(torchaudio.__version__)

/opt/conda/envs/HPM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA_VISIBLE_DEVICES: 7
CUDA Available: True
Device Count: 1
Current CUDA Device: 0
Using device: cuda:0
1.9.0+cu111 1 NVIDIA A100-SXM4-40GB
1.9.0+cu111
0.9.0


In [1]:
# TextGrid를 연다 
# sentence interval을 가져온다 
# xmin/xmax를 가져온다 
# frame_rate에 따라 start_frame, end_frame을 구한다. 

In [64]:
# textGridPath = glob.glob('/workspace/DATA/chem/sentences/*.TextGrid')
root_path = '/workspace/DATA'
speaker_name = 'chem'
root_path = os.path.join(root_path, speaker_name)
textGridPath = glob.glob(os.path.join(root_path,'sentences/*.TextGrid'))
base_name = 'Sh42Wpvs42k'
lip_embed = np.load(os.path.join(root_path,f'processed/mouth_emb/chem-face-{base_name}_face.npz'))

In [ ]:
textGridPath

In [60]:
# lip_embedding을 열어본다. 
lip_embedding = np.load('/workspace/DATA/chem/processed/mouth_emb/chem-face-_56-KofIBng_face.npz')
#/workspace/DATA/chem/preprocessed_test/duration/chem-_56-KofIBng-sentence-0.9-duration.npy
# lip_lens = np.array([lip_e.shape[1] for lip_e in lip_embedding['data']])
# lip_embedding = [lip_e[0] for lip_e in lip_embedding['data']]
# frame수와 일치하는지 본다. 
# 일치하면 frame 수대로 자르면 되는데 
# 이미지 리스트는 /workspace/DATA/chem/mouth 여기 있다.
folder_path_list = glob.glob('/workspace/DATA/chem/mouth/*_face')
folder = '/workspace/DATA/chem/mouth/_56-KofIBng_face'
image_list = glob.glob(folder + '/*.jpg')
tg_path = '/workspace/DATA/chem/sentences/_56-KofIBng.TextGrid'
# 이미지 파일 리스트를 가져옵니다.
folder = '/workspace/DATA/chem/mouth/_56-KofIBng_face'
image_list = glob.glob(os.path.join(folder, '*.jpg'))
base_name_list = [int(os.path.basename(name).split('.')[0]) for name in image_list]
textgrid = tgt.io.read_textgrid(tg_path)
sentences_tier = textgrid.get_tier_by_name("sentences")
frame_late = 25
for sentence in sentences_tier._objects:
    start_time = sentence.start_time
    end_time = sentence.end_time
    # 시작 프레임과 끝 프레임의 인덱스를 찾습니다.
    start_frame_idx = np.searchsorted(base_name_list, int(start_time * 25), side='left')
    end_frame_idx = np.searchsorted(base_name_list, int(end_time * 25), side='right')

    # 해당 문장의 lip embedding을 추출합니다.
    sentence_lip_embedding = lip_embedding['data'][0][start_frame_idx:end_frame_idx]

# 추출한 lip embedding을 저장합니다.
    output_path = f"/workspace/DATA/chem/test_2/{'_56-KofIBng_text'}-sentence-{start_time}-mouth_emb.npy"
    np.save(output_path, sentence_lip_embedding)

In [46]:
# lip_embedding을 열어본다. 
lip_embedding = np.load('/workspace/DATA/chem/processed/mouth_emb/chem-face-_7s29Q76st0_face.npz')
#mouth frame수와 일치하는지 본다. 
folder = '/workspace/DATA/chem/mouth/_7s29Q76st0_face'
image_list = glob.glob(folder + '/*.jpg')
base_name_List = [int(os.path.basename(name).split('.')[0]) for name in image_list]
print(f"frame num : {len(base_name_List)}  , lip_embedding_num: {lip_embedding['data'].shape}")


frame num : 4374  , lip_embedding_num: (1, 4374, 512)


In [50]:
lip_embedding['data'][0][0:3].shape

(3, 512)

In [54]:
lip_embedding['data'][0][0:4].shape

(4, 512)